In [1]:
import pandas as pd


def load_embeddings(file_path):
    embeddings = pd.read_csv(file_path)
    file_paths = embeddings["filepaths"]
    embeddings = embeddings.drop("filepaths", axis=1)
    return embeddings.values, file_paths

In [2]:
embeddings, image_paths = load_embeddings("embeddings/embeddings.csv")

In [3]:
from scipy.cluster.vq import kmeans2
from sklearn.decomposition import PCA
import numpy as np


def calculate_pca(embeddings, dim=16):
    print("Calculating PCA")
    pca = PCA(n_components=dim)
    pca_embeddings = pca.fit_transform(embeddings.squeeze())
    print("PCA calculating done!")
    return pca_embeddings


def calculate_kmeans(embeddings, k):
    print("KMeans processing...")
    centroid, labels = kmeans2(data=embeddings, k=k, minit="points")
    counts = np.bincount(labels)
    print("Kmeans done!")
    return centroid, labels

In [4]:
pca_dim = 16
cluster_range = 20
project_name = "animals"

pca_embeddings = calculate_pca(embeddings, dim=pca_dim)
centroids, labels = calculate_kmeans(pca_embeddings, k=cluster_range)

Calculating PCA


PCA calculating done!
KMeans processing...
Kmeans done!


In [5]:
import shutil
from itertools import compress
import os
from tqdm import tqdm

In [6]:
def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [7]:
for label_number in tqdm(range(cluster_range)):
    label_mask = labels == label_number

    path_images = list(compress(image_paths, label_mask))
    target_directory = f"./clusters/{project_name}/cluster_{label_number}"
    create_dir(target_directory)

    # Copy images into separate directories
    for img_path in path_images:
        shutil.copy2(img_path, target_directory)

  0%|          | 0/20 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: 'random_animal\\1000.jpeg'